# <center> Kim jesteśmy i dokąd zmierzamy (zawodowo)? </center>

<center> https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset </center>
<br>

In [1]:
# import bibliotek

import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
from math import pi

from jupyter_dash import JupyterDash

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display, clear_output

from bokeh.io import output_notebook, push_notebook, show 
from bokeh.plotting import figure, show#, output_file
from bokeh.models import NumeralTickFormatter
from bokeh.models import BoxZoomTool, PanTool, ResetTool, SaveTool, HoverTool
from bokeh.models import Span, Row, Column, CustomJS, DataTable, TableColumn, BoxAnnotation, Select
from bokeh.models import ColumnDataSource, CategoricalColorMapper
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.palettes import Category10_5, Category20_16
from bokeh.layouts import column, row, gridplot, grid, layout
from ipywidgets.embed import embed_minimal_html
from ipywidgets import Button, GridBox, Layout, ButtonStyle
from bokeh.transform import factor_cmap

output_notebook()

Loading BokehJS ...

**istalacja Jupyter dashboard**

conda install jupyter_dashboards -c conda-forge

In [2]:
# wczytanie danych

df = pd.read_csv('IBM_HR.csv')
df["Satisfaction"] = (df['JobSatisfaction'] + df['EnvironmentSatisfaction'] + df['RelationshipSatisfaction'])/3
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Satisfaction
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,80,0,8,0,1,6,4,0,5,2.333333
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,80,1,10,3,3,10,7,1,7,3.000000
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,80,0,7,3,3,0,0,0,0,3.000000
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,80,0,8,3,3,8,7,3,0,3.333333
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,80,1,6,3,3,2,2,2,2,2.333333


In [3]:
style = {'description_width': 'initial'}

# filtr department, role, level
widget_department = widgets.Dropdown(
    options = ['All departments'] + sorted(df['Department'].unique().tolist()),
    value = 'All departments',
    description = 'Department',
    style = style,
    disabled = False,
    layout = {'width': 'max-content'}
)

def update_job_role_handler(option_choice):
    if option_choice == 'All departments':
        widget_job_role_options = ['All available roles'] + sorted(df['JobRole'].unique().tolist())
    else:
        widget_job_role_options = ['All available roles'] +  sorted(df[df['Department'] == widget_department.value]['JobRole'].unique().tolist())
    widget_job_role.options = widget_job_role_options

    
# filtr job role
widget_job_role = widgets.Dropdown(
    options = ['All available roles'] + sorted(df['JobRole'].unique().tolist()),
    value ='All available roles',
    description ='Job role',
    style = style,
    disabled = False,
    layout = {'width': 'max-content'}
)

def update_job_level_handler(option_choice):
    if option_choice == 'All available roles':
        widget_job_level_options = ['All available job levels', 1, 2, 3, 4, 5]
    else:
        widget_job_level_options = ['All available job levels'] + sorted(df[df['JobRole'] == widget_job_role.value]['JobLevel'].unique().tolist())
    widget_job_level.options = widget_job_level_options

widget_job_level = widgets.Dropdown(
    options = ['All available job levels'] + sorted(df[df['JobRole'] == widget_job_role.value]['JobLevel'].unique().tolist()),
    value = 'All available job levels',
    description = 'Job level',
    style = style,
    disabled = False,
    layout = {'width': 'max-content'}
)

# filtr edukacja dyscyplina
widget_education_field = widgets.Dropdown(
    options = ['All education fields'] + sorted(df['EducationField'].unique().tolist()),
    value = 'All education fields',
    description = 'Education Field',
    style = style,
    disabled = False,
    layout = {'width': 'max-content'}
)
  
# filtr edukacja poziom ----> żeby wyświetlić słownie poziom edukacji dodać kolumnę i zastapić wartości stringami
widget_education_level = widgets.Dropdown(
    options = ["All education levels"] + sorted(df['Education'].unique().tolist()),
    value = "All education levels",
    description = 'Education Level',
    style = style,
    disabled = False,
    layout = {'width': 'max-content'}
)

# filtr Gender
widget_gender = widgets.Dropdown(
    options = ['Gender'] + sorted(df['Gender'].unique().tolist()),
    value = 'Gender',
    description = 'Gender',
    style = style,
    disabled = False,
    layout = {'width': 'max-content'}
)

# User input Monthly Income 
widget_monthly_income = widgets.IntSlider(
    min= df['MonthlyIncome'].min(),
    max = df['MonthlyIncome'].max(), 
    description='Monthly Income',
    style = style)

# User input Total Working Years
widget_total_years = widgets.IntSlider(
    min= df['TotalWorkingYears'].min(),
    max = df['TotalWorkingYears'].max(),
    description='Total Working Years',
    style = style)

# User input Department
widget_department_user = widgets.Dropdown(
    options = [''] + sorted(df['Department'].unique().tolist()),
    value = '',
    description = 'Department',
    style = style,
    disabled = False,
    layout = {'width': 'max-content'}
)

def update_job_role_user_handler(option_choice):
    widget_job_role_user_options = sorted(df[df['Department'] == widget_department_user.value]['JobRole'].unique().tolist())
    widget_job_role_user.options = widget_job_role_user_options

    
# User input job role
widget_job_role_user = widgets.Dropdown(
    options = [''] + sorted(df['JobRole'].unique().tolist()),
    value = '',
    description ='Job role',
    style = style,
    disabled = False,
    layout = {'width': 'max-content'}
)

def update_job_level_user_handler(option_choice):
    widget_job_level_user_options = sorted(df[df['JobRole'] == widget_job_role_user.value]['JobLevel'].unique().tolist())
    widget_job_level_user.options = widget_job_level_user_options

widget_job_level_user = widgets.Dropdown(
    options = [''] + sorted(df[df['JobRole'] == widget_job_role_user.value]['JobLevel'].unique().tolist()),
    value = '',
    description = 'Job level',
    style = style,
    disabled = False,
    layout = {'width': 'max-content'}
)

# User input edukacja dyscyplina
widget_education_field_user = widgets.Dropdown(
    options = [''] + sorted(df['EducationField'].unique().tolist()),
    value = '',
    description = 'Education Field',
    style = style,
    disabled = False,
    layout = {'width': 'max-content'}
)
  
# User input filtr edukacja poziom ----> żeby wyświetlić słownie poziom edukacji dodać kolumnę i zastapić wartości stringami
widget_education_level_user = widgets.Dropdown(
    options = [''] + sorted(df['Education'].unique().tolist()),
    value = '',
    description = 'Education Level',
    style = style,
    disabled = False,
    layout = {'width': 'max-content'}
)

# User input Gender
widget_gender_user = widgets.Dropdown(
    options = [''] + sorted(df['Gender'].unique().tolist()),
    value = '',
    description = 'Gender',
    style = style,
    disabled = False,
    layout = {'width': 'max-content'}
)

# User input Years at Company
widget_years_at_company = widgets.IntSlider(
    min= df['YearsAtCompany'].min(),
    max = df['YearsAtCompany'].max(),
    description='Years at Company',
    style = style)

# User input Work Life Balance
widget_work_life_balance = widgets.IntSlider(
    min= 0,
    max = df['WorkLifeBalance'].max(),
    description='Work Life Balance',
    style = style)

# User input Training
widget_training = widgets.IntSlider(
    min= 0,
    max = df['TrainingTimesLastYear'].max(),
    description='Training Times Last Year',
    style = style)


# User input Satisfaction
widget_satisfaction = widgets.FloatSlider(
    min= 0,
    max = df['Satisfaction'].max(),
    description='Satisfaction',
    step=1,
    style = style)
    
widgets.interactive_output(update_job_level_handler, {'option_choice': widget_job_role})
widgets.interactive_output(update_job_role_handler, {'option_choice': widget_department})

widgets.interactive_output(update_job_level_user_handler, {'option_choice': widget_job_role_user})
widgets.interactive_output(update_job_role_user_handler, {'option_choice': widget_department_user})


fliters = GridBox(children=[widget_department, widget_job_role, widget_job_level, 
                        widget_education_field, widget_education_level, widget_gender],
                layout=Layout(
                    width='100%',
                    grid_template_columns='300px 300px 300px',
                    grid_template_rows='auto auto auto',
                    grid_gap='5px 10px')
               )

user_inputs = GridBox(children=[widget_department_user, widget_job_role_user, widget_job_level_user, 
                                widget_education_field_user, widget_education_level_user, widget_gender_user, 
                                widget_monthly_income, widget_total_years, widget_years_at_company, 
                                widget_work_life_balance,widget_training, widget_satisfaction], 
                    layout=Layout(
                    width='100%',
                    grid_template_columns='300px 300px 300px',
                    grid_template_rows='auto auto auto auto',
                    grid_gap='5px 10px')
                     )

In [4]:
# widget handler
values = {'department_dropdown': widget_department.value,
            'job_role_dropdown': widget_job_role.value,
            'job_level_dropdown': widget_job_level.value,
            'education_field_dropdown': widget_education_field.value,
            'education_level_dropdown': widget_education_level.value,
            'gender_dropdown' : widget_gender.value,
            'monthly_income_slider' : widget_monthly_income.value,
            'total_years_slider' : widget_total_years.value,
            'department_dropdown_user': widget_department_user.value,
            'job_role_dropdown_user': widget_job_role_user.value,
            'job_level_dropdown_user': widget_job_level_user.value,
            'education_field_dropdown_user': widget_education_field_user.value,
            'education_level_dropdown_user': widget_education_level_user.value,
            'gender_dropdown_user' : widget_gender_user.value,
            'years_at_company_slider' : widget_years_at_company.value,
            'work_life_balance_slider': widget_work_life_balance.value,
            'training_slider': widget_training.value,
            'satisfaction_slider': widget_satisfaction.value,
         }
def widgets_handler(department_dropdown_val,
                    job_role_dropdown_val,
                    job_level_dropdown_val,
                    education_field_dropdown_val,
                    education_level_dropdown_val,
                    gender_dropdown_val, 
                    monthly_income_slider_val,
                    total_years_slider_val,
                    department_dropdown_user_val,
                    job_role_dropdown_user_val,
                    job_level_dropdown_user_val,
                    education_field_dropdown_user_val,
                    education_level_dropdown_user_val,
                    gender_dropdown_user_val,
                    years_at_company_slider_val,
                    work_life_balance_slider_val,
                    training_slider_val,
                    satisfaction_slider_val,
                   ):
    values['department_dropdown'] = department_dropdown_val
    values['education_level_dropdown'] = education_level_dropdown_val
    values['education_field_dropdown'] = education_field_dropdown_val
    values['job_role_dropdown'] = job_role_dropdown_val
    values['job_level_dropdown'] = job_level_dropdown_val
    values['gender_dropdown'] = gender_dropdown_val
    values['monthly_income_slider'] =  monthly_income_slider_val
    values['total_years_slider'] =  total_years_slider_val
    values['department_dropdown_user'] = department_dropdown_user_val
    values['education_level_dropdown_user'] = education_level_dropdown_user_val
    values['education_field_dropdown_user'] = education_field_dropdown_user_val
    values['job_role_dropdown_user'] = job_role_dropdown_user_val
    values['job_level_dropdown_user'] = job_level_dropdown_user_val
    values['gender_dropdown_user'] = gender_dropdown_user_val
    values['years_at_company_slider'] = years_at_company_slider_val
    values['work_life_balance_slider'] = work_life_balance_slider_val
    values['training_slider'] = training_slider_val
    values['satisfaction_slider'] = satisfaction_slider_val
    

########################################################################################

def df_filter():
    '''filtrowanie df z wykorzystaniem słownika values utworzonego z widgetów'''

    # filtrowanie departamentu
    if values['department_dropdown'] == 'All departments':
        filtr_dep = df['Department'].isin(df['Department'].unique())
    else:
        filtr_dep = df['Department'] == values['department_dropdown']

    # filtrowanie job role
    if values['job_role_dropdown'] == 'All available roles':
        filtr_role = df['JobRole'].isin(df['JobRole'].unique())
    else:
        filtr_role = df['JobRole'] == values['job_role_dropdown']

    # filtrowanie job level
    if values['job_level_dropdown'] == 'All available job levels':
        filtr_level = df['JobLevel'].isin(df['JobLevel'].unique())
    else:
        filtr_level = df['JobLevel'] == values['job_level_dropdown']

    # filtrowanie education field
    if values['education_field_dropdown'] == 'All education fields':
        filtr_edu_field = df['EducationField'].isin(df['EducationField'].unique())
    else:
        filtr_edu_field = df['EducationField'] == values['education_field_dropdown']

    # filtrowanie education level
    if values['education_level_dropdown'] == 'All education levels':
        filtr_edu_level = df['Education'].isin(df['Education'].unique())
    else:
        filtr_edu_level = df['Education'] == values['education_level_dropdown']    

    # filtrowanie gender
    if values['gender_dropdown'] == 'Gender':
        filtr_gender = df['Gender'].isin(df['Gender'].unique())
    else:
        filtr_gender = df['Gender'] == values['gender_dropdown']     
    
    df_filtred = df[(filtr_dep) & 
               (filtr_role) & 
               (filtr_level) & 
              (filtr_edu_field) & 
              (filtr_edu_level) & 
              (filtr_gender)]
    return df_filtred


def style(p):
    '''styl wykresów'''
    p.width = 350
    p.height = 350
    p.title.align = 'center'
    p.title.text_font_size = '14pt'
    p.xaxis.axis_label_text_font_size = '12pt'
    p.xaxis.major_label_text_font_size = '10pt'
    p.yaxis.axis_label_text_font_size = '12pt'
    p.yaxis.major_label_text_font_size = '10pt'
    p.axis.minor_tick_in = -3
    p.axis.minor_tick_out = 6
    p.toolbar_location='below'
    p.toolbar.autohide = True
    return p


def make_Age_chart(x, y):
    '''make a AGE bar chart'''
    # Set up the figure
    p1 = figure(x_range=(16,62), title='Age', x_axis_label='Age', y_axis_label='No of employees')

    # Add a vbar glyph
    p1.vbar(x=x, top=y, width=0.5, bottom=0, color='lightblue')

    # Add a style to the plot
    styled_p1 = style(p1)

    # Add the hover tool
    hover_1 = HoverTool(tooltips = [('Age', '@x'),
                                 ('No of employees', '@top')])
    styled_p1.add_tools(hover_1)
    return styled_p1


def make_Monthly_Income_histogram(histogram):
    '''make a HISTOGRAM Monthly Income'''

    income_hist, edges = histogram

    # DataFrame --> HoverTool
    income_df = pd.DataFrame({'count': income_hist, 'left': edges[:-1], 'right': edges[1:]})
    income_df['income_counts'] = ['%d employees' % count for count in income_df['count']]
    income_df['range'] = ['%d to %d $' % (left, right) for left, right in zip(income_df['left'], income_df['right'])]

    income_src = ColumnDataSource(income_df)

    # Set up the figure
    p2 = figure(x_range=(0, 21000), 
                title = 'Monthly Income', x_axis_label = 'Monthly Income', y_axis_label = 'Count')

    # Add a quad glyph
    p2.quad(bottom=0, top='count', left='left', right='right', source=income_src,
            fill_color='lightblue', line_color='black')
    p2.xaxis[0].formatter = NumeralTickFormatter(format='$0')

    # Add a style to the glyph
    styled_p2 = style(p2)

    # Add the hover tool
    hover_2 = HoverTool(tooltips = [('Range', '@range'),
                                 ('', '@income_counts')])
    styled_p2.add_tools(hover_2)

    # Add the user input
    user_value_2 = Span(location=values['monthly_income_slider'], dimension='height', line_color='red',
                                  line_width=3)
    p2.add_layout(user_value_2)
    return styled_p2


def make_Job_Level_chart(x, y):
    '''make a Job level chart'''
    
    # Set up the figure
    p3 = figure(x_range=(0,6), title='Job level', x_axis_label='Job level', y_axis_label='No of employees')

    
    # Add a vbar glyph
    p3.vbar(x=x, top=y, width=0.5, bottom=0, color='lightblue')
    p3.xaxis.ticker = [1,2,3,4,5,6]

    # Add a vbar glyp with user input
    if values['job_level_dropdown_user'] in range(1,6):
        user_value_3 = BoxAnnotation(left=values['job_level_dropdown_user']-0.25, right=values['job_level_dropdown_user']+0.25,
                                  bottom = 0, 
                                  top = df[df['JobLevel']==values['job_level_dropdown_user']]['JobLevel'].count(),
                                  fill_alpha=1,
                                  fill_color='orange')
        p3.renderers.extend([user_value_3])
    elif values['job_level_dropdown_user'] is None:
        p3.vbar(x=x, top=y, width=0.5, bottom=0, color='lightblue')


    # Add a style to the glyph
    styled_p3 = style(p3)
    
    #Add the hover tool
    hover_3 = HoverTool(tooltips = [('Job level', '@x'),
                                 ('No of employees', '@top')])
    styled_p3.add_tools(hover_3)        
    return styled_p3


def make_Performance_Rating_chart(x, y):
    '''make a Performance Rating bar chart'''
    # Set up the figure
    p4 = figure(x_range=(0,5), title='Performance rating', x_axis_label='Performance rating', y_axis_label='No of employees')

    # Add a vbar glyph
    p4.vbar(x=x, top=y, width=0.5, bottom=0, color='lightblue')
    p4.xaxis.ticker = [1,2,3,4,5]
    
    # Add a style to the plot
    styled_p4 = style(p4)

    #Add the hover tool
    hover_4 = HoverTool(tooltips = [('Performance Rating', '@x'),
                                 ('No of employees', '@top')])
    styled_p4.add_tools(hover_4)
    return styled_p4



def make_Distance_From_Home_chart(x, y):
    '''make a Distance from home bar chart'''
    # Set up the figure
    p5 = figure(x_range=(0,30), title='Distance from home', x_axis_label='Distance from home [km]', y_axis_label='No of employees')

    # Add a vbar glyph
    p5.vbar(x=x, top=y, width=0.5, bottom=0, color='lightblue')

    # Add a style to the plot
    styled_p5 = style(p5)

    #Add the hover tool
    hover_5 = HoverTool(tooltips = [('Distance from home', '@x'),
                                 ('No of employees', '@top')])
    styled_p5.add_tools(hover_5)
    return styled_p5


def make_Education_Field_chart(x, y):
    '''make a Education field bar chart'''
    # Set up the figure
    p6 = figure(x_range=x, title='Education field', x_axis_label='Education field', y_axis_label='No of employees')

    # Add a vbar glyph
    p6.vbar(x=x, top=y, width=0.5, bottom=0, color='lightblue')
    p6.xaxis.major_label_orientation = pi/4
    
    if values['education_field_dropdown_user'] == 'Human Resources':
        #p6.vbar(x=x, top=y, width=0.5, bottom=0, color='lightblue')
        p6.vbar(x=x, top=y, width=0.5, bottom=0,
                fill_color=factor_cmap('x', palette=['orange', 'lightblue','lightblue','lightblue','lightblue','lightblue'], 
                                       factors = sorted(df.EducationField.unique())))
        p6.xaxis.major_label_orientation = pi/4
    elif values['education_field_dropdown_user'] == 'Life Sciences':
        p6.vbar(x=x, top=y, width=0.5, bottom=0,
                fill_color=factor_cmap('x', palette=['lightblue', 'orange','lightblue','lightblue','lightblue','lightblue'], 
                                       factors = sorted(df.EducationField.unique())))
        p6.xaxis.major_label_orientation = pi/4
    elif values['education_field_dropdown_user'] == 'Marketing':
        p6.vbar(x=x, top=y, width=0.5, bottom=0,
                fill_color=factor_cmap('x', palette=['lightblue','lightblue', 'orange', 'lightblue','lightblue','lightblue'], 
                                       factors = sorted(df.EducationField.unique())))
        p6.xaxis.major_label_orientation = pi/4
    elif values['education_field_dropdown_user'] == 'Medical':
        p6.vbar(x=x, top=y, width=0.5, bottom=0,
                fill_color=factor_cmap('x', palette=['lightblue','lightblue','lightblue', 'orange','lightblue','lightblue'], 
                                       factors = sorted(df.EducationField.unique())))
        p6.xaxis.major_label_orientation = pi/4
    elif values['education_field_dropdown_user'] == 'Other':
        p6.vbar(x=x, top=y, width=0.5, bottom=0,
                fill_color=factor_cmap('x', palette=['lightblue','lightblue','lightblue','lightblue','orange','lightblue'], 
                                       factors = sorted(df.EducationField.unique())))
        p6.xaxis.major_label_orientation = pi/4
    elif values['education_field_dropdown_user'] == 'Technical Degree':
        p6.vbar(x=x, top=y, width=0.5, bottom=0,
                fill_color=factor_cmap('x', palette=['lightblue','lightblue','lightblue','lightblue','lightblue', 'orange'], 
                                       factors = sorted(df.EducationField.unique())))
        p6.xaxis.major_label_orientation = pi/4
    else:
        p6.vbar(x=x, top=y, width=0.5, bottom=0, color='lightblue')
        p6.xaxis.major_label_orientation = pi/4
       
    # Add a style to the plot
    styled_p6 = style(p6)

    #Add the hover tool
    hover_6 = HoverTool(tooltips = [('Education field', '@x'),
                                 ('No of employees', '@top')])
        
    styled_p6.add_tools(hover_6)
    return styled_p6


def make_Education_chart(x, y):
    '''make a Education chart'''
    
    # Set up the figure
    p7 = figure(x_range=(0,6), title='Education level', x_axis_label='Education level', y_axis_label='No of employees')
    p7.xaxis.ticker = [1,2,3,4,5,6]
    
    # Add a vbar glyp with user input
    if values['education_level_dropdown_user'] != '':
        user_value_7 = BoxAnnotation(left=values['education_level_dropdown_user']-0.25, 
                                    right=values['education_level_dropdown_user']+0.25,
                                    bottom = 0, 
                                    top = df[df['Education']==values['education_level_dropdown_user']]['Education'].count(),
                                    fill_alpha=1,
                                    fill_color='orange')
        p7.renderers.extend([user_value_7])
    else:
        p7.vbar(x=x, top=y, width=0.5, bottom=0, color='lightblue')

     # Add a vbar glyph
    p7.vbar(x=x, top=y, width=0.5, bottom=0, color='lightblue')   

    # Add a style to the glyph
    styled_p7 = style(p7)
    
    #Add the hover tool
    hover_7 = HoverTool(tooltips = [('Education level', '@x'),
                                 ('No of employees', '@top')])
    styled_p7.add_tools(hover_7)
    return styled_p7


def make_Training_Times_Last_Year_chart(x, y):
    '''make a Training times last year chart'''
    
    # Set up the figure
    p8 = figure(x_range=(0,8), title='Training times last year', x_axis_label='Training times last year', y_axis_label='No of employees')

    # Add a vbar glyph
    p8.vbar(x=x, top=y, width=0.5, bottom=0, color='lightblue')
    p8.xaxis.ticker = [1,2,3,4,5,6,7]

    # Add a style to the glyph
    styled_p8 = style(p8)
    
    #Add the hover tool
    hover_8 = HoverTool(tooltips = [('Training times last year', '@x'),
                                 ('No of employees', '@top')])
    styled_p8.add_tools(hover_8)
    
    # Add the user input
    user_value_8 = BoxAnnotation(left=values['training_slider']-0.25, right=values['training_slider']+0.25,
                                  bottom = 0, 
                                  top = df[df['TrainingTimesLastYear']==values['training_slider']]['TrainingTimesLastYear'].count(),
                                  fill_alpha=1,
                                  fill_color='orange')
    p8.renderers.extend([user_value_8])
    return styled_p8


def make_Years_At_Company_chart(x, y):
    '''make a Years at company chart'''
    
    # Set up the figure
    p9 = figure(x_range=(0,45), title='Years at Company', x_axis_label='Years at Company', y_axis_label='No of employees')

    # Add a vbar glyph
    p9.vbar(x=x, top=y, width=0.5, bottom=0, color='lightblue')

    # Add a style to the glyph
    styled_p9 = style(p9)
    
    #Add the hover tool
    hover_9 = HoverTool(tooltips = [('Years at Company', '@x'),
                                 ('No of employees', '@top')])
    styled_p9.add_tools(hover_9)
    
    # Add the user input
    user_value_9 = Span(location=values['years_at_company_slider'], dimension='height', line_color='red',
                                  line_width=3)
    p9.add_layout(user_value_9)
    return styled_p9
    

def make_Total_Working_Years_chart(x,y):
    '''make a Total working years chart '''
    
    # Set up the figure
    p10 = figure(x_range=(0,45), title='Total Working Years', x_axis_label='Total Working Years', y_axis_label='No of employees')

    # Add a vbar glyph
    p10.vbar(x=x, top=y, width=0.5, bottom=0, color='lightblue')

    # Add a style to the glyph
    styled_p10 = style(p10)
    
    #Add the hover tool
    hover_10 = HoverTool(tooltips = [('Total Working Years', '@x'),
                                 ('No of employees', '@top')])
    styled_p10.add_tools(hover_10)
    
    # Add the user input
    user_value_10 = Span(location=values['total_years_slider'], dimension='height', line_color='red',
                                  line_width=3)
    p10.add_layout(user_value_10)
    return styled_p10


def make_Work_Life_Balance_chart(x,y):
    '''make a Work Life Balance chart '''
    
    # Set up the figure
    p11 = figure(x_range=(0,5), title='Work Life Balance', x_axis_label='Work Life Balance', y_axis_label='No of employees')

    # Add a vbar glyph
    p11.vbar(x=x, top=y, width=0.5, bottom=0, color='lightblue')
    p11.xaxis.ticker = [1,2,3,4,5]

    # Add a style to the glyph
    styled_p11 = style(p11)
    
    #Add the hover tool
    hover_11 = HoverTool(tooltips = [('Work Life Balance', '@x'),
                                 ('No of employees', '@top')])
    styled_p11.add_tools(hover_11)
    
    # Add the user input
    user_value_11 = BoxAnnotation(left=values['work_life_balance_slider']-0.25, right=values['work_life_balance_slider']+0.25,
                                  bottom = 0, 
                                  top = df[df['WorkLifeBalance']==values['work_life_balance_slider']]['WorkLifeBalance'].count(),
                                  fill_alpha=1,
                                  fill_color='orange')
    p11.renderers.extend([user_value_11])
    return styled_p11


def make_Satisfaction_histogram(histogram):
    '''make a HISTOGRAM Satisfaction'''

    sat_hist, edges = histogram

    # DataFrame --> HoverTool
    sat_df = pd.DataFrame({'count': sat_hist, 'left': edges[:-1], 'right': edges[1:]})
    sat_df['sat_counts'] = ['%d employees' % count for count in sat_df['count']]
    sat_df['range'] = ['%d to %d' % (left, right) for left, right in zip(sat_df['left'], sat_df['right'])]

    sat_src = ColumnDataSource(sat_df)

    # Set up the figure
    p12 = figure(x_range=(0, 5), 
                title = 'Satisfaction', x_axis_label = 'Satisfaction', y_axis_label = 'No of employees')

    # Add a quad glyph
    p12.quad(bottom=0, top='count', left='left', right='right', source=sat_src,
            fill_color='lightblue', line_color='black')
    p12.xaxis.ticker = [1,2,3,4,5]
  
    # Add a style to the glyph
    styled_p12 = style(p12)

    # Add the hover tool
    hover_12 = HoverTool(tooltips = [('Range', '@range'),
                                 ('', '@sat_counts')])
    styled_p12.add_tools(hover_12)

    # Add the user input
    user_value_12 = Span(location=values['satisfaction_slider'], dimension='height', line_color='red',
                                  line_width=3)
    p12.add_layout(user_value_12)
    return styled_p12


def make_Business_Travel_chart(x, y):
    '''make a Business Travel bar chart'''
    # Set up the figure
    p13 = figure(x_range=x, title='Business Travel', x_axis_label='Business Travel', y_axis_label='No of employees')

    # Add a vbar glyph
    p13.vbar(x=x, top=y, width=0.5, bottom=0, color='lightblue')
    p13.xaxis.major_label_orientation = pi/4

    # Add a style to the plot
    styled_p13 = style(p13)

    #Add the hover tool
    hover_13 = HoverTool(tooltips = [('Business Travel', '@x'),
                                 ('No of employees', '@top')])
    styled_p13.add_tools(hover_13)
    
    return styled_p13


def table(source, title_x, field_y, title_y):
    columns = [TableColumn(field ="index",  title = title_x),
           TableColumn(field = field_y,  title = title_y)]
    
    table = DataTable(source = source, columns = columns, width =300, #height = 200, 
                      margin = (5, 5, 0, 50), 
                      autosize_mode = 'fit_columns', index_position = None, row_height = 20)
    return table


def make_grid():
    '''make a GRID of charts'''
    df_filtred = df_filter()
    p1 = make_Age_chart(x = np.sort(df_filtred['Age'].unique()), 
                        y = df_filtred['Age'].value_counts().sort_index())
    table_1 = table(ColumnDataSource(pd.DataFrame(df_filtred['Age'].describe().map(lambda x: f"{x:0.2f}"))), 
                    "", "Age", "Age Statistics")
    
    p2 = make_Monthly_Income_histogram(histogram = np.histogram(df_filtred['MonthlyIncome'], bins = 20))
    table_2 = table(ColumnDataSource(pd.DataFrame(df_filtred['MonthlyIncome'].describe().map(lambda x: f"{x:0.2f}"))), 
                    "", "MonthlyIncome", "Monthly Income Statistics")
    
    p3 = make_Job_Level_chart(x = np.sort(df_filtred['JobLevel'].unique()), 
                              y = df_filtred['JobLevel'].value_counts().sort_index())
    table_3 = table(ColumnDataSource(pd.DataFrame(df_filtred['JobLevel'].describe().map(lambda x: f"{x:0.2f}"))), 
                    "", "JobLevel", "Job Level Statistics")
    
    p4 = make_Performance_Rating_chart(x = np.sort(df_filtred['PerformanceRating'].unique()), 
                        y = df_filtred['PerformanceRating'].value_counts().sort_index())
    table_4 = table(ColumnDataSource(pd.DataFrame(df_filtred['PerformanceRating'].describe().map(lambda x: f"{x:0.2f}"))), 
                    "", "PerformanceRating", "Performance Rating Statistics")
    
    p5 = make_Distance_From_Home_chart(x = np.sort(df_filtred['DistanceFromHome'].unique()), 
                        y = df_filtred['DistanceFromHome'].value_counts().sort_index())
    table_5 = table(ColumnDataSource(pd.DataFrame(df_filtred['DistanceFromHome'].describe().map(lambda x: f"{x:0.2f}"))), 
                    "", "DistanceFromHome", "Distance From Home Statistics")
    
    p6 = make_Education_Field_chart(x = np.sort(df_filtred['EducationField'].unique()), 
                        y = df_filtred['EducationField'].value_counts().sort_index())
    table_6 = table(ColumnDataSource(pd.DataFrame(df_filtred['EducationField'].describe())), 
                    "", "EducationField", "Education Field Statistics")
    
    p7 = make_Education_chart(x = np.sort(df_filtred['Education'].unique()), 
                              y = df_filtred['Education'].value_counts().sort_index())
    table_7 = table(ColumnDataSource(pd.DataFrame(df_filtred['Education'].describe().map(lambda x: f"{x:0.2f}"))), 
                    "", "Education", "Education Statistics")
    
    p8 = make_Training_Times_Last_Year_chart(x = np.sort(df_filtred['TrainingTimesLastYear'].unique()), 
                              y = df_filtred['TrainingTimesLastYear'].value_counts().sort_index())
    table_8 = table(ColumnDataSource(pd.DataFrame(df_filtred['TrainingTimesLastYear'].describe().map(lambda x: f"{x:0.2f}"))), 
                    "", "TrainingTimesLastYear", "Training Times Last Year Statistics")
    
    p9 = make_Years_At_Company_chart(x = np.sort(df_filtred['YearsAtCompany'].unique()), 
                              y = df_filtred['YearsAtCompany'].value_counts().sort_index())
    table_9 = table(ColumnDataSource(pd.DataFrame(df_filtred['YearsAtCompany'].describe().map(lambda x: f"{x:0.2f}"))), 
                    "", "YearsAtCompany", "Years At Company Statistics")
    
    p10 = make_Total_Working_Years_chart(x = np.sort(df_filtred['TotalWorkingYears'].unique()), 
                              y = df_filtred['TotalWorkingYears'].value_counts().sort_index())
    table_10 = table(ColumnDataSource(pd.DataFrame(df_filtred['TotalWorkingYears'].describe().map(lambda x: f"{x:0.2f}"))), 
                    "", "TotalWorkingYears", "Total Working Years Statistics")
    
    p11= make_Work_Life_Balance_chart(x = np.sort(df_filtred['WorkLifeBalance'].unique()),  
                              y = df_filtred['WorkLifeBalance'].value_counts().sort_index())
    table_11 = table(ColumnDataSource(pd.DataFrame(df_filtred['WorkLifeBalance'].describe().map(lambda x: f"{x:0.2f}"))), 
                    "", "WorkLifeBalance", "Work Life Balance Statistics")
    
    p12 = make_Satisfaction_histogram(histogram = np.histogram(df_filtred['Satisfaction'], bins = 5))
    table_12 = table(ColumnDataSource(pd.DataFrame(df_filtred['Satisfaction'].describe().map(lambda x: f"{x:0.2f}"))), 
                    "", "Satisfaction", "Satisfaction Statistics")
    
    p13= make_Business_Travel_chart(x = np.sort(df_filtred['BusinessTravel'].unique()),  
                              y = df_filtred['BusinessTravel'].value_counts().sort_index())
    table_13 = table(ColumnDataSource(pd.DataFrame(df_filtred['BusinessTravel'].describe())), 
                    "", "BusinessTravel", "Business Travel Statistics")
    
    grid_1 = grid([p1, p2, p3, table_1, table_2, table_3,
                  p6, p7, p8, table_6, table_7, table_8,
                  p4, p5, p9, table_4, table_5, table_9,
                  p10, p11, p12, table_10, table_11, table_12,
                  p13, None, None, table_13],
                  ncols=3, sizing_mode = 'stretch_height'
                 )

    #return output_file("foo.html")  # na razie okno zewnętrzne wyrzucone
    show(grid_1, notebook_handle=True)

         
def main():
    widgets_handler(department_dropdown_val = widget_department.value,
                    education_level_dropdown_val = widget_education_level.value,
                    education_field_dropdown_val = widget_education_field.value,
                    job_role_dropdown_val = widget_job_role.value,
                    job_level_dropdown_val = widget_job_level.value,
                    gender_dropdown_val = widget_gender.value, 
                    monthly_income_slider_val = widget_monthly_income.value,
                    total_years_slider_val = widget_total_years.value,
                    department_dropdown_user_val = widget_department_user.value,
                    education_level_dropdown_user_val = widget_education_level_user.value,
                    education_field_dropdown_user_val = widget_education_field_user.value,
                    job_role_dropdown_user_val = widget_job_role_user.value,
                    job_level_dropdown_user_val = widget_job_level_user.value,
                    gender_dropdown_user_val = widget_gender_user.value,
                    years_at_company_slider_val = widget_years_at_company.value,
                    work_life_balance_slider_val = widget_work_life_balance.value,
                    training_slider_val = widget_training.value,
                    satisfaction_slider_val = widget_satisfaction.value
                   )
    make_grid()   


In [5]:
# top=df['WorkLifeBalance'][values['work_life_balance_slider']].
# top

**OVERALL STATISTICS FOR THE COMPANY**

In [6]:
# Button START
run_button = widgets.Button(description="Start")

text_1 = """Please select filters:"""

text_2 = "\nPlease choose your profile:"

print(text_1)
display(fliters)
print(text_2)
display(user_inputs)  
display(run_button)


Please select filters:


GridBox(children=(Dropdown(description='Department', layout=Layout(width='max-content'), options=('All departm…


Please choose your profile:


GridBox(children=(Dropdown(description='Department', layout=Layout(width='max-content'), options=('', 'Human R…

Button(description='Start', style=ButtonStyle())

In [7]:
def button_callback(button):
    clear_output()
    print(text_1)
    display(fliters)
    print(text_2)
    display(user_inputs)  
    display(run_button) 
    main()
    

run_button.on_click(button_callback)

In [8]:
main()